In [29]:
import os
import openai
import sys
import numpy as np

In [30]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY_']
HUGGINGFACE_TOKEN = os.environ['HUGGINGFACE_TOKEN']


In [31]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_function = SentenceTransformer(model_name)

In [ ]:
from langchain.vectorstores import Chroma
persist_directory = 'db/chroma'

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

## Document Loading and Splitting

In [34]:
from langchain.document_loaders import PyPDFLoader

In [35]:
loader = PyPDFLoader('data/ReAct.pdf')
pages = loader.load()

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 100
)

In [37]:
splits = splitter.split_documents(pages)
len(splits)

73

In [38]:
splits[0].page_content

'Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, while actions allow it to 

In [39]:
temp = embedding_function.encode(splits[0].page_content)
print(temp)

[-1.42610213e-02 -1.00450464e-01  5.63753508e-02 -4.76643853e-02
  3.94125842e-02  6.60193060e-03  6.64848685e-02  5.35975285e-02
  9.59211141e-02  3.18779238e-02 -3.51582319e-02 -3.34922858e-02
  5.00640534e-02  2.25127488e-02  1.10441603e-01 -1.65781528e-02
  1.40233627e-02 -3.54926032e-03 -1.04830660e-01 -7.76339397e-02
  6.04142994e-02 -2.94648502e-02  3.89858894e-02 -3.81193310e-02
 -1.00444049e-01 -2.74950191e-02  2.60499828e-02 -7.76545629e-02
  5.04011922e-02  1.95956845e-02  6.54101297e-02  4.33976427e-02
  1.21673066e-02  1.39522394e-02 -8.21192637e-02  1.36251256e-01
 -5.80241233e-02  2.52280006e-04 -1.92695986e-02 -9.64747369e-02
 -8.16828236e-02 -3.09433434e-02  5.04717380e-02 -5.84152602e-02
  9.27214026e-02 -5.27057275e-02 -3.17007080e-02 -8.79268919e-04
 -7.07282647e-02  3.39548034e-03 -1.18072145e-01 -1.58097241e-02
  1.02372235e-02 -1.07866814e-02  8.86400230e-03  5.31580951e-03
  2.22819168e-02  4.35798541e-02  2.89247930e-02 -8.27872008e-02
 -9.46392938e-02 -9.24950

## Create Embeddings and store them in DB

In [40]:
embeddings = []
documents = []
metadata = []
ids = []
count = 0
# .encode("hello how are you")
for split in splits:
    embeddings.append(list((embedding_function.encode(split.page_content).astype(float))))
    documents.append(split.page_content)
    metadata.append(split.metadata)
    ids.append(f"id{count}")
    count += 1

    

# print(len(embeddings))

In [41]:
print(len(documents))

73


## Creating VectorDB from Chroma

In [42]:
import chromadb
chroma_client = chromadb.Client()

In [43]:
collection = chroma_client.create_collection(name='ReActPaper')

In [44]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [63]:
query = "What does it do about hotpotQA and Wikipedia API"
results = collection.query(
    query_texts=query,
    n_results=5
)
len(results['documents'])

1

In [62]:
print(f"""Answer to the above query: {query} is \n {str(results['documents'][0][0])}""")

Answer to the above query: What does it do about hotpotQA and Wikipedia API is 
 demonstrating a synergy of reasoning and acting.
3.1 S ETUP
Domains We consider two datasets challenging knowledge retrieval and reasoning: (1) Hot-
PotQA (Yang et al., 2018), a multi-hop question answering benchmark that requires reasoning
over two or more Wikipedia passages, and (2) FEVER (Thorne et al., 2018), a fact veriﬁcation
benchmark where each claim is annotated SUPPORTS, REFUTES, or NOT ENOUGH INFO, based
on if there exists a Wikipedia passage to verify the claim. In this work, we operate in a question-only
setup for both tasks, where models only receive the question/claim as input without access to support
paragraphs, and have to rely on their internal knowledge or retrieve knowledge via interacting with
an external environment to support reasoning.
Action Space We design a simple Wikipedia web API with three types of actions to support
interactive information retrieval: (1) search [entity ], wh

In [73]:
for doc in results['documents']:
    print(doc)
    print(len(results['documents']))
# print(**results['documents'])

['Published as a conference paper at ICLR 2023\nMethod Pick Clean Heat Cool Look Pick 2 All\nAct (best of 6) 88 42 74 67 72 41 45\nReAct (avg) 65 39 83 76 55 24 57\nReAct (best of 6) 92 58 96 86 78 41 71\nReAct-IM (avg) 55 59 60 55 23 24 48\nReAct-IM (best of 6) 62 68 87 57 39 33 53\nBUTLER g(best of 8) 33 26 70 76 17 12 22\nBUTLER (best of 8) 46 39 74 100 22 24 37\nTable 3: AlfWorld task-speciﬁc success rates (%). BUTLER and\nBUTLER gresults are from Table 4 of Shridhar et al. (2020b). All\nmethods use greedy decoding, except that BUTLER uses beam search.Method Score SR\nAct 62.3 30.1\nReAct 66.6 40.0\nIL 59.9 29.1\nIL+RL 62.4 28.7\nHuman82.1 59.6Expert\nTable 4: Score and suc-\ncess rate (SR) on Web-\nshop. IL/IL+RL taken\nfrom Yao et al. (2022).\ntrained with 1,012 human annotated trajectories, and a imitation + reinforcement learning (IL + RL)\nmethod additionally trained with 10,587 training instructions.\nResults ReAct outperforms Act on both ALFWorld (Table 3) and Webshop (Table

## Using LangChain library to build a vector db from the same set of documents

In [3]:
# todo: Check below Code to integrate Chroma using LangChain
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [4]:

# load the document and split it into chunks
loader = PyPDFLoader("data/ReAct.pdf")
documents = loader.load()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)



In [5]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/ibnabeeali/miniconda3/envs/LangChain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [71]:

# query it
query = "What does it do about hotpotQA and Wikipedia API"
docs = db.similarity_search(query, k=3)
print(f"""Total Documents Returned: {len(docs)}""")

# print results
print(docs[2].page_content)

Total Documents Returned: 3
Published as a conference paper at ICLR 2023
We conduct empirical evaluations of ReAct and state-of-the-art baselines on four diverse benchmarks:
question answering (HotPotQA, Yang et al., 2018), fact veriﬁcation (Fever, Thorne et al., 2018),
text-based game (ALFWorld, Shridhar et al., 2020b), and webpage navigation (WebShop, Yao
et al., 2022). For HotPotQA and Fever, with access to a Wikipedia API that the model can interact
with, ReAct outperforms vanilla action generation models while being competitive with chain-of-
thought reasoning ( CoT) (Wei et al., 2022). The best approach overall is a combination of ReAct
andCoT that allows for the use of both internal knowledge and externally obtained information
during reasoning. On ALFWorld and WebShop, two or even one-shot ReAct prompting is able
to outperform imitation or reinforcement learning methods trained with 103∼105task instances,
with an absolute improvement of 34% and 10% in success rates respectively

In [74]:
string = f"{'*'*50}"
count = 0
similar_docs = db.similarity_search(query, k=3)
query_docs = []
for doc in similar_docs:
    query_docs.append(string)
    query_docs.append(f'doc{count+1}')
    query_docs.append(doc.page_content)
    query_docs.append(string)
    count+=1
    
documents = ''.join(query_docs)

**************************************************doc1demonstrating a synergy of reasoning and acting.
3.1 S ETUP
Domains We consider two datasets challenging knowledge retrieval and reasoning: (1) Hot-
PotQA (Yang et al., 2018), a multi-hop question answering benchmark that requires reasoning
over two or more Wikipedia passages, and (2) FEVER (Thorne et al., 2018), a fact veriﬁcation
benchmark where each claim is annotated SUPPORTS, REFUTES, or NOT ENOUGH INFO, based
on if there exists a Wikipedia passage to verify the claim. In this work, we operate in a question-only
setup for both tasks, where models only receive the question/claim as input without access to support
paragraphs, and have to rely on their internal knowledge or retrieve knowledge via interacting with
an external environment to support reasoning.
Action Space We design a simple Wikipedia web API with three types of actions to support
interactive information retrieval: (1) search [entity ], which returns the ﬁrst 5 sent

## Integrate an LLM via API to test the qualitative performance of the model

In [68]:
import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [80]:
def prompt(query):
    
    string = f"{'*'*50}"
    count = 0
    similar_docs = db.similarity_search(query, k=3)
    query_docs = []
    for doc in similar_docs:
        query_docs.append(string)
        query_docs.append(f'doc{count+1}')
        query_docs.append(doc.page_content)
        query_docs.append(string)
        count+=1
        
    documents = ''.join(query_docs)

    prompt = f"""Given the user query: '{query}', and the relevant document chunks: {documents},
      your task is to generate an informative response based solely on the information contained within the 
      provided document chunks."""
    
    output = query({
	"inputs": prompt,
    })
    return output


In [82]:

result = prompt("When was this paper published and at which conference")


TypeError: 'str' object is not callable